In [ ]:
import pandas as pd
import numpy as np
import warnings
from pandas import ExcelWriter
warnings.filterwarnings("ignore")

pd.pandas.set_option("display.max_rows",None)

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
data = pd.read_excel("/content/drive/MyDrive/hospital.xlsx")

In [ ]:
data.head()

,FAMILY-ID,User id,Start-Date,hospitaltype,T-duration,P-date,Referred,procedure,cost,claim-duration,claim-status
0,1,1,2020-01-11,public,7,2020-01-12,NO,2,3000,7,SUCCESS
1,1,1,2020-01-11,public,7,2020-01-15,NO,6,4000,7,SUCCESS
2,1,1,2020-02-21,public,1,2020-01-22,NaN,5,0,4,PENDING
3,1,2,2020-02-24,private,2,2020-02-24,NO,4,6000,10,PENDING
4,1,2,2020-02-25,private,2,2020-02-26,NO,3,5000,3,SUCCESS


In [ ]:
data.rename(columns = {"User id":"id"},inplace=True)

In [ ]:
data["procedure"] = data["procedure"].astype("str")

In [ ]:
output = pd.DataFrame(columns=["id","procedure"])
for i,j in data.groupby(["id","Start-Date"]):
  col1 = j.iloc[0]["id"]
  col2 = j["procedure"].str.cat(sep = ",")
  output = output.append({"id":col1,"procedure":col2},ignore_index = True)
print(output.head())

  id procedure
0  1       2,6
1  1         5
2  2         4
3  2         3
4  2         1


In [ ]:
unique_val = data["procedure"].unique()
len(unique_val)

6

##Sequence Generation Task1

In [ ]:
from itertools import permutations
values = []

for i in range(1,len(unique_val)+1):
  if i<=3:
    perm = permutations(unique_val,i)
    values.append(perm)

In [ ]:
perms = []
for i in values:
  for j in i:
    perms.append(j)

In [ ]:
len(perms)

156

##Desired Output is stored in output1

In [ ]:
def convert(x):
  return x.split(",")

In [ ]:
output["procedure"] = output["procedure"].map(lambda x : convert(x))

In [ ]:
output.head()

,id,procedure
0,1,"[2, 6]"
1,1,[5]
2,2,[4]
3,2,[3]
4,2,[1]


In [ ]:
output1 = output.groupby("id")["procedure"].apply(tuple).reset_index()

In [ ]:
output1.head()

,id,procedure
0,1,"([2, 6], [5])"
1,2,"([4], [3], [1])"
2,3,"([3], [2], [4])"
3,4,"([1, 4], [3])"
4,5,"([5], [6], [5])"


In [ ]:
#Original Data
data1 = pd.DataFrame(output["procedure"])
patients = len(data1)

In [ ]:
output1.head()

,id,procedure
0,1,"([2, 6], [5])"
1,2,"([4], [3], [1])"
2,3,"([3], [2], [4])"
3,4,"([1, 4], [3])"
4,5,"([5], [6], [5])"


##Getting Count of Sequences of Task1 present in data_count variable


In [ ]:
seq = []
for i in range(len(output1["procedure"])):
  a = str(output1["procedure"][i]).replace('(','').replace(')','').replace("[","").replace("]","").replace("'","")
  a = a.replace(" ","")
  a = (a.split(","))
  seq.append(a)

In [ ]:
seq

[['2', '6', '5'],
 ['4', '3', '1'],
 ['3', '2', '4'],
 ['1', '4', '3'],
 ['5', '6', '5'],
 ['1', '2', '4'],
 ['6', '3', '4'],
 ['2', '3', '4'],
 ['2', '6', '5'],
 ['1', '3', '5'],
 ['6', '1', '2'],
 ['3', '5', '6']]

In [ ]:
from itertools import combinations
fii = []
for i in range(len(seq)):
  for j in range(1,4):
    comb = combinations(seq[i],j)
    fii.append(comb)

In [ ]:
final_seq = []
for i in fii:
  for j in i:
    final_seq.append(j)

In [ ]:
ones = np.ones(len(final_seq)).astype("int")

In [ ]:
data_final = pd.DataFrame({"key":final_seq,"count":ones},columns = ["key","count"])

In [ ]:
data_final = data_final.groupby("key").sum().reset_index()
data_final["key"].head()

0         (1,)
1       (1, 2)
2    (1, 2, 4)
3       (1, 3)
4    (1, 3, 5)
Name: key, dtype: object

In [ ]:
print(len(perms))
data_comb = list(data_final["key"])
len(data_comb)

156


39

In [ ]:
other= []
for element in data_comb:
  if element in perms:
    perms.remove(element)
  else:
    print(element)

('5', '5')
('5', '6', '5')


In [ ]:
other = perms
len(other)

119

In [ ]:
#Getting their count
zeros = np.zeros(len(other)).astype("int")

In [ ]:
frame = pd.DataFrame({"key": other,"count":zeros})

In [ ]:
frame.head()

,key,count
0,"(2, 1)",0
1,"(5, 2)",0
2,"(5, 4)",0
3,"(5, 3)",0
4,"(5, 1)",0


In [ ]:
data_count = pd.concat([data_final,frame],axis = 0,ignore_index=True)
data_count.head()

,key,count
0,"(1,)",5
1,"(1, 2)",2
2,"(1, 2, 4)",1
3,"(1, 3)",2
4,"(1, 3, 5)",1


In [ ]:
len(data_final),len(frame)
data_count.shape

(158, 2)

In [ ]:
data_final

,key,count
0,"(1,)",5
1,"(1, 2)",2
2,"(1, 2, 4)",1
3,"(1, 3)",2
4,"(1, 3, 5)",1
5,"(1, 4)",2
6,"(1, 4, 3)",1
7,"(1, 5)",1
8,"(2,)",6
9,"(2, 3)",1


In [ ]:
data_final.loc[data_final["key"] == ("2, 6")]

,key,count


##Association Rules 

##Support

In [ ]:
#Calculating the support for all the sequences
support = []
element = []
sequence = []
n = len(output1)

for i in range(len(data_final)):
  a = str(data_final["key"][i]).replace("(","").replace(")","").replace("'","").replace(",","")
  for j in unique_val:
    for k in range(len(data_final)):
      b = str(data_final["key"][k]).replace("(","").replace(")","").replace("'","").replace(",","")
      c = a + " " +j
      if a != j:
        if c in b:
          d = data_final["count"].where(data_final["key"] == tuple(map(str, c.split(' '))))
          d = d.dropna()
          element.append(a)
          sequence.append(j)
          support.append(round(d.iloc[0]/n,4))
          break

In [ ]:
final = pd.DataFrame({"element":element,"sequence":sequence,"support":support})

In [ ]:
#Calculation used for support of X and Support of Y
element_support = []
sequence_support = []
n = len(output1)

k = 0
for i in range(len(element)):
  a = element[i]
  c = sequence[k]
  k += 1
  flag = flag1 = False

  for j in range(len(data_final)):
    b = str(data_final["key"][j]).replace("(","").replace(")","").replace("'","").replace(",","")
    if a == b:
      element_support.append(data_final["count"][j]/n)
      flag = True
    
    if c == b:
      sequence_support.append(data_final["count"][j]/n)
      flag1 = True
    
    if flag == True and flag1 == True:
      break

##Confidence

In [ ]:
#Calculating the confidence for all the sequences
final["confidence"] = round(final["support"]/pd.Series(element_support),4)

##Lift

In [ ]:
#Calculation for Lift
n = len(output1)
num = final["support"]
denom = pd.Series(element_support) * pd.Series(sequence_support)
fin = num/denom
final["lift"] = round(fin,4)

In [ ]:
final

,element,sequence,support,confidence,lift
0,1,2,0.1667,0.4001,0.8002
1,1,5,0.0833,0.1999,0.3998
2,1,4,0.1667,0.4001,0.8002
3,1,3,0.1667,0.4001,0.6859
4,1 2,4,0.0833,0.4998,0.9996
5,1 3,5,0.0833,0.4998,0.9996
6,1 4,3,0.0833,0.4998,0.8568
7,2,6,0.1667,0.3334,0.6668
8,2,5,0.1667,0.3334,0.6668
9,2,4,0.2500,0.5000,1.0000


In [ ]:
data_final.loc[data_final["key"] == ('5',)]

,key,count
27,"(5,)",6


In [ ]:
P = pd.Series(element_support) * len(output1)
N = pd.Series(sequence_support) * len(output1)

In [ ]:
p = []
n = []
for i in range(len(final)):
  a = final["element"][i]
  b = final["sequence"][i]
  count = 0
  count1 = 0
  for j in range(len(final)):
    if a == final["element"][j]:
      count += 1
    if b == final["sequence"][j]:
      count1 += 1
  p.append(count)
  n.append(count1)

In [ ]:
p = pd.Series(p)
n = pd.Series(n)

In [ ]:
final["g"] = round(p/(p+n+2),3)
final["wLap"] = round((p+1)*(P+N)/(p+n+2)*P,3)
final["LS"] = round(p*N/(n*P),3)
final["Rss"] = round((p/P) - (n/N),3)
final["MS"] = round(p/(n+P),3)

In [ ]:
n[2]

8

In [ ]:
Cohen = ((P+N)*(p/(p+n))) - P / (((P+N)/2) * ((p+n+P)/(p+n))) - P
Coleman = (((P+N) * (p/(p+n))) - P) / N

In [ ]:
final["C1"] = round(Coleman * (2 + Cohen)/3,3)
final["C2"] = round(Coleman * 0.5*(1+(p/P)),3)

In [ ]:
final

,element,sequence,support,confidence,lift,g,wLap,LS,Rss,MS,C1,C2
0,1,2,0.1667,0.4001,0.8002,0.400,27.500,1.200,0.133,0.444,0.054,0.075
1,1,5,0.0833,0.1999,0.3998,0.308,21.154,0.686,-0.367,0.333,-0.021,-0.150
2,1,4,0.1667,0.4001,0.8002,0.286,19.643,0.600,-0.533,0.308,-0.002,-0.200
3,1,3,0.1667,0.4001,0.6859,0.364,27.273,1.120,0.086,0.400,0.029,0.043
4,1 2,4,0.0833,0.4998,0.9996,0.091,2.909,0.375,-0.833,0.100,-0.030,-0.139
5,1 3,5,0.0833,0.4998,0.9996,0.100,3.200,0.429,-0.667,0.111,-0.033,-0.125
6,1 4,3,0.0833,0.4998,0.8568,0.125,4.500,0.700,-0.214,0.143,-0.028,-0.054
7,2,6,0.1667,0.3334,0.6668,0.400,36.000,1.000,0.000,0.400,0.000,0.000
8,2,5,0.1667,0.3334,0.6668,0.308,27.692,0.571,-0.500,0.308,0.026,-0.227
9,2,4,0.2500,0.5000,1.0000,0.286,25.714,0.500,-0.667,0.286,0.074,-0.278


In [ ]:
final.to_excel("/content/drive/MyDrive/new_rules.xlsx",index = False)

In [ ]:
conf_thresh = 0.4; lift_thresh = 1.0
filter = pd.DataFrame([final.loc[i] for i in range(len(final)) if final["confidence"][i] >= conf_thresh and final["lift"][i] >= lift_thresh]).reset_index(drop = True)

In [ ]:
filter

,element,sequence,support,confidence,lift,g,wLap,LS,Rss,MS,C1,C2
0,2,4,0.2500,0.5000,1.0000,0.286,25.714,0.500,-0.667,0.286,0.074,-0.278
1,2 3,4,0.0833,0.9996,1.9992,0.091,1.273,0.750,-0.333,0.111,-0.019,-0.037
2,2 6,5,0.1667,1.0002,2.0004,0.100,3.200,0.429,-0.667,0.111,-0.033,-0.125
3,3 2,4,0.0833,0.9996,1.9992,0.091,1.273,0.750,-0.333,0.111,-0.019,-0.037
4,4 3,1,0.0833,0.4998,1.1995,0.143,4.000,0.625,-0.300,0.167,-0.040,-0.090
5,6,5,0.2500,0.5000,1.0000,0.357,30.857,0.714,-0.333,0.385,-0.019,-0.153
6,6 1,2,0.0833,0.9996,1.9992,0.143,2.000,1.500,0.333,0.200,0.048,0.067
7,6 3,4,0.0833,0.9996,1.9992,0.091,1.273,0.750,-0.333,0.111,-0.019,-0.037
